In [ ]:
import pandas as pd
import osmnx as ox

CIDADE = "Recife, Pernambuco, Brasil"
ARQUIVO = "bairros_vizinhos.csv"
GEO_PACKAGE = "grafo_recife.graphml"
SAIDA = "bairros_vizinhos_tratados.csv"

mapeamento_pesos = {
    "motorway": 0.8,
    "motorway_link": 0.8,
    "trunk": 1.0,
    "trunk_link": 1.0,
    "primary": 1.2,
    "primary_link": 1.2,
    "secondary": 1.5,
    "secondary_link": 1.5,
    "tertiary": 1.8,
    "tertiary_link": 1.8,
    "residential": 2.0,
    "service": 2.5,
    "living_street": 2.5,
    "pedestrian": 3.0,
    "track": 3.0,
}

tipo_normalizado = {
"motorway":	"Via expressa / Rodovia",
"motorway_link": "Alça de acesso à via expressa",
"trunk": "Via arterial / Rodovia urbana",
"trunk_link": "Acesso à via arterial",
"primary":	"Avenida principal",
"primary_link":	"Alça de acesso à avenida principal",
"secondary": "Avenida secundária	",
"secondary_link": "Ramal de ligação secundária",
"tertiary": "Rua de ligação / Coletora local",
"tertiary_link": "Trecho curto entre ruas terciárias",
"residential": "Rua local / de bairro",
"service": "Rua de serviço / interna",
"living_street": "Rua compartilhada / de pedestres",
"pedestrian": "Calçadão / rua exclusiva de pedestres",
"track": "Estrada de terra / rural",
}

def normalizar_tipo(tipo):
    if isinstance(tipo, list):
        prioridade = ["motorway", "trunk", "primary", "secondary", "tertiary", "residential", "service"]
        for p in prioridade:
            if p in tipo:
                return p
        return tipo[0]
    if "_link" in tipo:
        return tipo.replace("_link", "")
    return tipo




In [11]:
# === Lê os pares de bairros ===
df = pd.read_csv(ARQUIVO, sep=",")
df.head()


,name,neighbor
0,Aflitos,Rosarinho
1,Aflitos,Encruzilhada
2,Aflitos,Espinheiro
3,Aflitos,Graças
4,Jaqueira,Tamarineira


In [9]:
print("Carregando mapa da cidade a partir do OSM...")
G = ox.graph_from_place(CIDADE, network_type="drive")


Carregando mapa da cidade a partir do OSM...


In [10]:
import osmnx as ox
from concurrent.futures import ProcessPoolExecutor, as_completed

def processar_par(row):
    bairro_a = row["name"]
    bairro_b = row["neighbor"]

    try:
        orig_node = ox.distance.nearest_nodes(G, *ox.geocode(f"{bairro_a}, {CIDADE}")[::-1])
        dest_node = ox.distance.nearest_nodes(G, *ox.geocode(f"{bairro_b}, {CIDADE}")[::-1])

        route = ox.routing.shortest_path(G, orig_node, dest_node, weight="length")

        nome_selecionado = tipo_selecionado = id_rua = None

        for nodo_origem, nodo_destino in zip(route[:-1], route[1:]):
            atributos_aresta = G.edges[nodo_origem, nodo_destino, 0]
            nome = atributos_aresta.get("name")
            tipo = atributos_aresta.get("highway")
            osmid = atributos_aresta.get("osmid")

            if isinstance(nome, list):
                nome = nome[0]
            if isinstance(tipo, list):
                tipo = tipo[0]
            if isinstance(osmid, list):
                osmid = osmid[0]

            if nome:
                nome_selecionado = nome
                tipo_selecionado = tipo
                id_rua = osmid
                break

        if nome_selecionado is None:
            nome_selecionado = "N/A"
            tipo_selecionado = "N/A"
            id_rua = "N/A"

        return {
            "Bairro": bairro_a,
            "Vizinho": bairro_b,
            "Logradouro": nome_selecionado,
            "Tipo": tipo_selecionado,
            "Peso": mapeamento_pesos.get(normalizar_tipo(tipo_selecionado), 2.0),
            "Tipo Normalizado": tipo_normalizado.get(normalizar_tipo(tipo_selecionado), "Desconhecido"),
            "Id Rua": f"https://openstreetmap.org/way/{id_rua}",
        }

    except Exception as e:
        print(f"Erro ao processar {bairro_a}-{bairro_b}: {e}")
        return {
            "Bairro": bairro_a,
            "Vizinho": bairro_b,
            "Logradouro": "N/A",
            "Tipo": "N/A",
            "Peso": "N/A",
            "Tipo Normalizado": "N/A",
            "Id Rua": "N/A",
        }

# execução paralela
total_size = len(df)
print(f"Total de pares a processar: {total_size}")

resultados = []
with ProcessPoolExecutor() as executor:
    futures = {executor.submit(processar_par, row): i for i, row in df.iterrows()}
    for i, future in enumerate(as_completed(futures), 1):
        resultado = future.result()
        resultados.append(resultado)
        print(f"[{i}/{total_size}] {resultado['Bairro']} ↔ {resultado['Vizinho']} processado")


Total de pares a processar: 494
[1/494] Jaqueira ↔ Santana processado
[2/494] Jaqueira ↔ Tamarineira processado
[3/494] Hipódromo ↔ Encruzilhada processado
[4/494] Aflitos ↔ Rosarinho processado
[5/494] Jaqueira ↔ Parnamirim processado
[6/494] Aflitos ↔ Encruzilhada processado
[7/494] Aflitos ↔ Graças processado
[8/494] Jaqueira ↔ Torre processado
[9/494] Aflitos ↔ Espinheiro processado
[10/494] Jaqueira ↔ Graças processado
[11/494] Hipódromo ↔ Campo Grande processado
[12/494] Ponto de Parada ↔ Encruzilhada processado
[13/494] Ponto de Parada ↔ Campo Grande processado
[14/494] Ponto de Parada ↔ Arruda processado
[15/494] Hipódromo ↔ Ponto de Parada processado
[16/494] Rosarinho ↔ Aflitos processado
[17/494] Ponto de Parada ↔ Rosarinho processado
[18/494] Ponto de Parada ↔ Hipódromo processado
[19/494] Rosarinho ↔ Ponto de Parada processado
[20/494] Rosarinho ↔ Encruzilhada processado
[21/494] Rosarinho ↔ Tamarineira processado
[22/494] Tamarineira ↔ Jaqueira processado
[23/494] Tamarin

KeyboardInterrupt: 

In [ ]:
df_result = pd.DataFrame(resultados)
df_result.to_csv(SAIDA, index=False)
print(f"✅ Resultado salvo em {SAIDA}")
df_result.head()


✅ Resultado salvo em /home/miguel/workspace/projeto_grafos/data/bairros_vizinhos_tratados.csv


,Bairro,Vizinho,Logradouro,Tipo,Peso,Tipo Normalizado,Id Rua,Comentário OSM
0,Jaqueira,Graças,Rua Neto de Mendonça,tertiary,1.8,Rua de ligação / Coletora local,https://openstreetmap.org/way/965984036,Specify road smoothness
1,Aflitos,Encruzilhada,Rua Manuel de Carvalho,tertiary,1.8,Rua de ligação / Coletora local,https://openstreetmap.org/way/134174443,Specify road smoothness
2,Hipódromo,Encruzilhada,Rua Carlos Fernandes,tertiary,1.8,Rua de ligação / Coletora local,https://openstreetmap.org/way/140478431,Acréscimo de nova rota cicloviária
3,Hipódromo,Campo Grande,Rua Carlos Fernandes,tertiary,1.8,Rua de ligação / Coletora local,https://openstreetmap.org/way/974688528,Acréscimo de nova rota cicloviária
4,Jaqueira,Santana,Rua Neto de Mendonça,tertiary,1.8,Rua de ligação / Coletora local,https://openstreetmap.org/way/965984036,Specify road smoothness


In [ ]:
bairros_finais = set(df_result['Bairro'])

bairros_originals = set(df['name'])

diferenca = bairros_originals - bairros_finais
print(len(diferenca))
for bairro in diferenca:
    print("Bairro não encontrado na primeira coluna:", bairro)
    
    
    
bairros_finais = set(df_result['Vizinho'])

bairros_originals = set(df['neighbor'])

diferenca = bairros_originals - bairros_finais
print(len(diferenca))
for bairro in diferenca:
    print("Bairro não encontrado na segunda coluna:", bairro)


9
Bairro não encontrado na primeira coluna: Ipsep
Bairro não encontrado na primeira coluna: Soledade
Bairro não encontrado na primeira coluna: Mangabeira
Bairro não encontrado na primeira coluna: Poço da Panela
Bairro não encontrado na primeira coluna: Água Fria
Bairro não encontrado na primeira coluna: Parnamirim
Bairro não encontrado na primeira coluna: Santo Antônio
Bairro não encontrado na primeira coluna: Zumbi
Bairro não encontrado na primeira coluna: Oitenta
16
Bairro não encontrado na segunda coluna: Alto José do Pinho
Bairro não encontrado na segunda coluna: Jaqueira
Bairro não encontrado na segunda coluna: Bairro do Recife
Bairro não encontrado na segunda coluna: Cajueiro
Bairro não encontrado na segunda coluna: Caxangá
Bairro não encontrado na segunda coluna: Cohab
Bairro não encontrado na segunda coluna: Alto do Mandu
Bairro não encontrado na segunda coluna: Aflitos
Bairro não encontrado na segunda coluna: Hipódromo
Bairro não encontrado na segunda coluna: Coqueiral
Bairro 

Versão 12 por umbraosmbr em 2024-09-30T23:57:57Z
Changeset: 157317632
Comentário: inclusão de area verde, correção de caminho, inclusão de casas no bairro do corrego do jenipapo, recife/PE #UMBRAOSM #Mapatona #SenamaSoftwarelivreBrasil2024
------------------------------------------------------------
